In [1]:
from sqlalchemy.ext.asyncio import create_async_engine
import os
from dotenv import load_dotenv

_ = load_dotenv()

username = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST", "localhost")
database = os.getenv("DB_NAME")

# Create the connection string
connection_string = f"mysql+aiomysql://{username}:{password}@{host}/{database}"

# Create an engine instance
engine = create_async_engine(connection_string)

try:
    # Test the connection
    async with engine.connect() as connection:
        print("Connection successful!")
except Exception as e:
    print(f"Error connecting to database: {e}")

Connection successful!


In [1]:
# Add these cells to your existing notebook

# Cell 1: Enhanced connection with query capabilities
from sqlalchemy.ext.asyncio import create_async_engine
from sqlalchemy import text
import os
from dotenv import load_dotenv

_ = load_dotenv()

username = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST", "localhost")
database = os.getenv("DB_NAME")

connection_string = f"mysql+aiomysql://{username}:{password}@{host}/{database}"
engine = create_async_engine(connection_string)


async def run_query(query_string):
    """Helper function to run queries"""
    try:
        async with engine.connect() as connection:
            result = await connection.execute(text(query_string))
            return result.fetchall()
    except Exception as e:
        print(f"Error executing query: {e}")
        return None


# Cell 2: Show all tables
async def show_tables():
    tables = await run_query("SHOW TABLES")
    if tables:
        print("📊 Available Tables:")
        for table in tables:
            print(f"  - {table[0]}")
    return tables


await show_tables()


# Cell 3: View logs table structure
async def describe_logs():
    structure = await run_query("DESCRIBE logs")
    if structure:
        print("🔍 Logs Table Structure:")
        for row in structure:
            print(f"  {row[0]} | {row[1]} | {row[2]} | {row[3]}")


await describe_logs()


# Cell 4: View recent logs
async def view_recent_logs(limit=10):
    logs = await run_query(f"SELECT * FROM logs ORDER BY created_at DESC LIMIT {limit}")
    if logs:
        print(f"📋 Recent {len(logs)} Logs:")
        for i, log in enumerate(logs, 1):
            print(f"\n--- Log {i} ---")
            print(f"ID: {log[0]}")
            print(f"Session: {log[1]}")
            print(f"User Input: {log[2][:100]}...")
            print(f"Agent Output: {log[3][:100]}...")
            print(f"Start Time: {log[4]}")
            print(f"End Time: {log[5]}")
            print(f"Created: {log[6]}")


await view_recent_logs()


# Cell 5: Interactive query runner
async def run_custom_query():
    query = input("Enter your SQL query: ")
    result = await run_query(query)
    if result:
        for row in result:
            print(row)


# Uncomment to use:
# await run_custom_query()

📊 Available Tables:
  - logs
🔍 Logs Table Structure:
  id | int | NO | PRI
  session_id | varchar(255) | NO | MUL
  user_input | text | NO | 
  agent_output | longtext | NO | 
  start_time | datetime | YES | MUL
  end_time | datetime | YES | 
  created_at | datetime | YES | MUL
📋 Recent 2 Logs:

--- Log 1 ---
ID: 2
Session: dev
User Input: Analyze the financial reports and transcripts for the last three quarters and provide a qualitative ...
Agent Output: {"messages": ["content=\"Analyze the financial reports and transcripts for the last three quarters a...
Start Time: 2025-08-02 23:48:18
End Time: 2025-08-02 23:48:39
Created: 2025-08-02 18:18:39

--- Log 2 ---
ID: 1
Session: dev
User Input: Hi...
Agent Output: {"messages": ["content='Hi' additional_kwargs={} response_metadata={} id='794d8126-f7ee-4391-b32f-b6...
Start Time: 2025-08-02 22:07:36
End Time: 2025-08-02 22:07:37
Created: 2025-08-02 16:37:37
